# Najniebezpieczniejsze czynniki wypadków w danej dzielnicy NY(wersja 1)

## Poprzez analizę, które czynniki występowały najczęściej w danej dzielnicy Nowego Yorku, uzyskamy informację jakie czyniki są najniebezpieczniejsze dla określonej dzielnicy powodując największą ilość wypadków.

In [2]:
import sqlite3
import numpy

import pandas as pd
import requests

## Pobranie danych

In [3]:
nypd_df = pd.read_csv("/home/student/PythonDeveloper/pythonProject/Projekt_Pandas/data/nypd-motor-vehicle-collisions(1).csv")#, usecols=["name", "ingredients"]
nypd_df

/tmp/ipykernel_6861/2504016720.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_df = pd.read_csv("/home/student/PythonDeveloper/pythonProject/Projekt_Pandas/data/nypd-motor-vehicle-collisions(1).csv")#, usecols=["name", "ingredients"]


,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30T00:00:00.000,21:53,BRONX,10460.0,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016.0,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612173,2012-07-21T00:00:00.000,22:46,BROOKLYN,11226.0,40.651741,-73.965241,"{'type': 'Point', 'coordinates': [-73.9652409,...",PARADE PLACE,CROOKE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,147595,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN
1612174,2012-07-09T00:00:00.000,16:45,NaN,NaN,NaN,NaN,NaN,EDWARD L GRANT HIGHWAY,WEST 169 STREET,NaN,...,Fatigued/Drowsy,Unspecified,NaN,NaN,85220,PASSENGER VEHICLE,LIVERY VEHICLE,PASSENGER VEHICLE,NaN,NaN
1612175,2012-07-09T00:00:00.000,14:35,NaN,NaN,40.791492,-73.935763,"{'type': 'Point', 'coordinates': [-73.935763, ...",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2853431,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1612176,2012-07-18T00:00:00.000,9:20,MANHATTAN,10024.0,40.779479,-73.973573,"{'type': 'Point', 'coordinates': [-73.9735729,...",CENTRAL PARK WEST,WEST 77 STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,52405,SPORT UTILITY / STATION WAGON,VAN,BUS,NaN,NaN


##### Wczytanie tabeli ze wszyskimi danymi do przeprowadzenia analizy

## Wydzielenie kolumny z dzielnicami NY i uzyskanie wartości unikalnych 

In [4]:
borough_nypd_df= nypd_df[['BOROUGH']]
borough_nypd_df.drop_duplicates()

,BOROUGH
0,QUEENS
1,BROOKLYN
2,MANHATTAN
3,BRONX
7,NaN
18,STATEN ISLAND


##### Tabela przedstawia nazwy dzielnic, które będziemy analizować 

## Usunięcie wierszy nie posiadających wartości z kolumny BOROUGH

In [6]:
nypd_df_clean = nypd_df.dropna(subset=['BOROUGH'])
nypd_df_clean

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30T00:00:00.000,21:53,BRONX,10460.0,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016.0,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612171,2012-07-14T00:00:00.000,18:30,BROOKLYN,11216.0,40.680089,-73.943973,"{'type': 'Point', 'coordinates': [-73.9439725,...",FULTON STREET,BROOKLYN AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,179501,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
1612172,2012-07-08T00:00:00.000,22:30,QUEENS,11368.0,40.742096,-73.867473,"{'type': 'Point', 'coordinates': [-73.8674729,...",JUNCTION BOULEVARD,CORONA AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,255043,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1612173,2012-07-21T00:00:00.000,22:46,BROOKLYN,11226.0,40.651741,-73.965241,"{'type': 'Point', 'coordinates': [-73.9652409,...",PARADE PLACE,CROOKE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,147595,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN
1612176,2012-07-18T00:00:00.000,9:20,MANHATTAN,10024.0,40.779479,-73.973573,"{'type': 'Point', 'coordinates': [-73.9735729,...",CENTRAL PARK WEST,WEST 77 STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,52405,SPORT UTILITY / STATION WAGON,VAN,BUS,NaN,NaN


##### Tabela z danym, w której usunęliśmy wiersze nie posiadające wartości w kolumnie BOROUGHT 

## Uzyskanie tabeli z danymi dotyczącymi pierwszej analizowanej dzielnicy: Queens

In [5]:
nypd_df_clean_queens = nypd_df_clean[
    nypd_df_clean['BOROUGH'].str.contains('queens', case=False)
]
nypd_df_clean_queens

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
6,2019-08-31T00:00:00.000,2:16,QUEENS,11434.0,40.667522,-73.780630,"{'type': 'Point', 'coordinates': [-73.78063, 4...",ROCKAWAY BOULEVARD,NORTH CONDUIT AVENUE,NaN,...,NaN,NaN,NaN,NaN,4198571,Sedan,NaN,NaN,NaN,NaN
8,2019-08-02T00:00:00.000,16:30,QUEENS,11434.0,40.666492,-73.765360,"{'type': 'Point', 'coordinates': [-73.76536, 4...",SOUTH CONDUIT AVENUE,179 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4181743,Pick-up Truck,Pick-up Truck,NaN,NaN,NaN
14,2019-08-12T00:00:00.000,11:20,QUEENS,11420.0,40.678550,-73.814220,"{'type': 'Point', 'coordinates': [-73.81422, 4...",NaN,NaN,125-05 115 AVENUE,...,Unsafe Speed,NaN,NaN,NaN,4187599,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
28,2019-08-20T00:00:00.000,11:00,QUEENS,11105.0,40.773663,-73.902390,"{'type': 'Point', 'coordinates': [-73.90239, 4...",21 AVENUE,42 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4192979,Sedan,Sedan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612151,2012-07-03T00:00:00.000,12:50,QUEENS,11422.0,40.665559,-73.729843,"{'type': 'Point', 'coordinates': [-73.7298428,...",SUNRISE HIGHWAY,HOOK CREEK BOULEVARD,NaN,...,Unspecified,NaN,NaN,NaN,219426,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
1612158,2012-07-22T00:00:00.000,0:50,QUEENS,11106.0,40.756256,-73.924504,"{'type': 'Point', 'coordinates': [-73.9245037,...",36 STREET,35 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,272811,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1612166,2012-07-20T00:00:00.000,18:00,QUEENS,11101.0,40.747482,-73.941350,"{'type': 'Point', 'coordinates': [-73.9413498,...",JACKSON AVENUE,43 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,238841,PASSENGER VEHICLE,OTHER,NaN,NaN,NaN
1612172,2012-07-08T00:00:00.000,22:30,QUEENS,11368.0,40.742096,-73.867473,"{'type': 'Point', 'coordinates': [-73.8674729,...",JUNCTION BOULEVARD,CORONA AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,255043,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


##### Tablea dotycząca tylko dzielnicy Queens

## Wydzielenie kolumn z czynnikami z ramki dotyczącej Queens

In [6]:
queens_factors=nypd_df_clean_queens.iloc[:,18:23]
queens_factors

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5
0,Passing Too Closely,Unspecified,NaN,NaN,NaN
6,Unspecified,NaN,NaN,NaN,NaN
8,Reaction to Uninvolved Vehicle,Unspecified,NaN,NaN,NaN
14,View Obstructed/Limited,Unsafe Speed,NaN,NaN,NaN
28,Unspecified,Unspecified,NaN,NaN,NaN
...,...,...,...,...,...
1612151,Unspecified,Unspecified,NaN,NaN,NaN
1612158,Unspecified,Unspecified,NaN,NaN,NaN
1612166,Unspecified,Unspecified,NaN,NaN,NaN
1612172,Other Electronic Device,Unspecified,NaN,NaN,NaN


##### Tabela dotycząca tylko czynników ryzyka z dzielnicy Queens

## Określenie jak często dany czynnik występuje w danej kolumnie z czynnikami

In [7]:
queens_factors_numbers = queens_factors[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].apply(pd.Series.value_counts) 
queens_factors_numbers

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5
1,3,NaN,NaN,NaN,NaN
80,32,5.0,NaN,NaN,NaN
Accelerator Defective,160,11.0,1.0,NaN,NaN
Aggressive Driving/Road Rage,915,165.0,12.0,2.0,NaN
Alcohol Involvement,3306,192.0,15.0,3.0,2.0
...,...,...,...,...,...
Unspecified,115699,226337.0,18909.0,4115.0,1100.0
Using On Board Navigation Device,12,NaN,NaN,NaN,NaN
Vehicle Vandalism,10,1.0,NaN,NaN,NaN
View Obstructed/Limited,1961,420.0,2.0,NaN,NaN


##### Tabela przedstawia jak często określony czynnik pojawia się w kolumnie dotyczącej danego samochodu

## Kolejnymi krokami jest zsumowanie oraz posortowanie czynników ze wszystkich kolumn

In [12]:
queens_factors_numbers['Sum'] = queens_factors_numbers[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].sum(axis=1)
queens_factors_numbers_sort=queens_factors_numbers.sort_values(by='Sum', ascending=False)
queens_factors_numbers_sort

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,Sum
Unspecified,115699,226337.0,18909.0,4115.0,1100.0,366160.0
Driver Inattention/Distraction,60223,13303.0,152.0,23.0,3.0,73704.0
Failure to Yield Right-of-Way,24398,3534.0,18.0,3.0,NaN,27953.0
Backing Unsafely,15254,1510.0,17.0,3.0,NaN,16784.0
Following Too Closely,9270,1156.0,56.0,8.0,NaN,10490.0
...,...,...,...,...,...,...
Vehicle Vandalism,10,1.0,NaN,NaN,NaN,11.0
Eating or Drinking,5,1.0,NaN,NaN,NaN,6.0
1,3,NaN,NaN,NaN,NaN,3.0
Listening/Using Headphones,2,NaN,NaN,NaN,NaN,2.0


##### Tabela z czynnikami dla dzielnicy Queens zsumowanymi dla wszystkich samochodów oraz posortowanymi od najczęsciej do najrzadziej występujących

In [13]:
queens_factors_numbers_sort.head(2)

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,Sum
Unspecified,115699,226337.0,18909.0,4115.0,1100.0,366160.0
Driver Inattention/Distraction,60223,13303.0,152.0,23.0,3.0,73704.0


##### Tabela z dwoma najczęstszymi czynnikami w dzielnicy Queens 

# ROZWIĄZANIE: 
## Czynnikiem, który spowodował największą ilość wypadków w dzielnicy Queens jest nieuwaga/rozproszenie kierowcy. 
## W tabeli pokazano dwie największe wartości, aby była możliwość pominięcia czynniki niezdefiniowanego, w zależności od potrzeb.

## Wykonanie kodu w wersji skróconej dla dzielnicy Brooklyn

In [13]:
nypd_df_clean_brooklyn = nypd_df_clean[
    nypd_df_clean['BOROUGH'].str.contains('brooklyn', case=False)
]
brooklyn_factors=nypd_df_clean_brooklyn.iloc[:,18:23]
brooklyn_factors_numbers = brooklyn_factors[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].apply(pd.Series.value_counts) 
brooklyn_factors_numbers['Sum'] = brooklyn_factors_numbers[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].sum(axis=1)
brooklyn_factors_numbers_sorted=brooklyn_factors_numbers.sort_values(by='Sum', ascending=False)
brooklyn_factors_numbers_sorted['Sum'].head(2)

Unspecified                       449875.0
Driver Inattention/Distraction     63866.0
Name: Sum, dtype: float64

##### Przedstawienie dwóch najczęstszych czynników dla dzielnicy Brooklyn 

## Wykonanie kodu w wersji skróconej dla dzielnicy Manhattan

In [14]:
nypd_df_clean_manhattan = nypd_df_clean[
    nypd_df_clean['BOROUGH'].str.contains('manhattan', case=False)
]
manhattan_factors=nypd_df_clean_manhattan.iloc[:,18:23]
manhattan_factors_numbers = manhattan_factors[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].apply(pd.Series.value_counts) 
manhattan_factors_numbers['Sum'] = manhattan_factors_numbers[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].sum(axis=1)
manhattan_factors_numbers_sorted=manhattan_factors_numbers.sort_values(by='Sum', ascending=False)
manhattan_factors_numbers_sorted['Sum'].head(2)

Unspecified                       280938.0
Driver Inattention/Distraction     71360.0
Name: Sum, dtype: float64

##### Przedstawienie dwóch najczęstszych czynników dla dzielnicy Manhattan

## Wykonanie kodu w wersji skróconej dla dzielnicy Bronx

In [15]:
nypd_df_clean_bronx = nypd_df_clean[
    nypd_df_clean['BOROUGH'].str.contains('bronx', case=False)
]
bronx_factors=nypd_df_clean_bronx.iloc[:,18:23]
bronx_factors_numbers = bronx_factors[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].apply(pd.Series.value_counts) 
bronx_factors_numbers['Sum'] = bronx_factors_numbers[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].sum(axis=1)
bronx_factors_numbers_sorted=bronx_factors_numbers.sort_values(by='Sum', ascending=False)
bronx_factors_numbers_sorted['Sum'].head(2)

Unspecified                       192380.0
Driver Inattention/Distraction     29952.0
Name: Sum, dtype: float64

##### Przedstawienie dwóch najczęstszych czynników dla dzielnicy Bronx

## Wykonanie kodu w wersji skróconej dla dzielnicy State Island

In [15]:
nypd_df_clean_staten_island = nypd_df_clean[
    nypd_df_clean['BOROUGH'].str.contains('staten island', case=False)
]
staten_island_factors=nypd_df_clean_staten_island.iloc[:,18:23]
staten_island_factors_numbers = staten_island_factors[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].apply(pd.Series.value_counts) 
staten_island_factors_numbers['Sum'] = staten_island_factors_numbers[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']].sum(axis=1)
staten_island_factors_numbers_sorted= staten_island_factors_numbers.sort_values(by='Sum', ascending=False)
staten_island_factors_numbers_sorted['Sum'].head(2)

Unspecified                       62921.0
Driver Inattention/Distraction    10541.0
Name: Sum, dtype: float64

##### Przedstawienie dwóch najczęstszych czynników dla dzielnicy Staten Island

# Podsumowanie: 
## Na podstawie analizy otrzymanych danych możemy wnioskować, że czynnikiem, który pojawiał się najczęściej podczas wypadków samochodowych, a tym samym jest najniebezpieczniejszy była nieuwaga lub rozproszenie kierowcy.  